In [8]:
# Configurações básicas do algoritmo
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import pandas as pd
import os
from datetime import datetime

# Definir o nome do arquivo a ser tratado
# IMPORTANTE: Inserir o nome do arquivo
nome_do_arquivo = 'ArquivoBruto_ControleControle_31052024'

# ID do último registro contido no banco
valorInicial = 6515592

parametroData = nome_do_arquivo.split("_")[2]
operation = nome_do_arquivo.split("_")[1]

# Caminho dos arquivos
caminho_do_arquivo = f'/content/drive/MyDrive/TIMrotinasPython/RetornoBruto/{nome_do_arquivo}.csv'
caminho_da_tabela_comparativa = '/content/drive/MyDrive/TIMPythonTabelasComparativa/tabelaComparativaStatus.csv'
caminho_da_tabela_ddd = '/content/drive/MyDrive/TIMPythonTabelasComparativa/tabelaDDD.csv'
caminho_da_tabela_MOP = '/content/drive/MyDrive/TIMPythonTabelasComparativa/tabelaMOPparaPython.csv'

# Importa a tabela bruta com o Pandas, selecionando somente as colunas que serão utilizadas
tabelaBruta = pd.read_csv(caminho_do_arquivo, sep=';', low_memory=False, on_bad_lines='skip')

qtd_tabelaBruta = len(tabelaBruta)
print(f'A tabela Bruta possui {qtd_tabelaBruta}')

tabelaFiltrada = tabelaBruta[["gravacao", "historico_tipo_descricao", "ddd", "telefone", "usuario_gravou_codigo_descricao"]]

# Importa a tabela de comparação da tabulação com status da tabulação
tabelaComparativa = pd.read_csv(caminho_da_tabela_comparativa, sep=';', encoding='ISO-8859-1')
tabelaComparativa = tabelaComparativa[["tabulacao", "status"]]

# Importa a tabela de comparação do DDD com a região
tabelaDDD = pd.read_csv(caminho_da_tabela_ddd, sep=';', encoding='ISO-8859-1')
tabelaDDD = tabelaDDD[["cd_ddd", "cd_regiao"]]

# Importa a tabela de comparação do nome do operador com a matricula
tabelaMOP = pd.read_csv(caminho_da_tabela_MOP, sep=';', encoding='ISO-8859-1')
tabelaMOP = tabelaMOP[["nomeOperador", "matricula"]]

# Filtra a coluna DDD retirando as linhas que não possuem números
filtro = tabelaFiltrada['ddd'].notnull()
tabelaSemDDDnulo = tabelaFiltrada.loc[filtro].copy()

qtd_tabelaSemDDDnulo = len(tabelaSemDDDnulo)
print(f'A tabela teve filtro de excluindo os campos sem DDD, ficando com a quantidade de: {qtd_tabelaSemDDDnulo} registros')

# Alteração do tipo de dado das colunas DDD e Telefone
tabelaSemDDDnulo['ddd'] = tabelaSemDDDnulo['ddd'].astype(int)
tabelaSemDDDnulo['telefone'] = tabelaSemDDDnulo['telefone'].astype(int)

tabelaSemDDDnulo = tabelaSemDDDnulo.copy()

# Divide a coluna "gravacao" em "Data" e "Hora"
tabelaSemDDDnulo.loc[:,"data"] = pd.to_datetime(tabelaSemDDDnulo["gravacao"], dayfirst=True).dt.date
tabelaSemDDDnulo.loc[:,"hora"] = pd.to_datetime(tabelaSemDDDnulo["gravacao"], dayfirst=True).dt.time

# Excluir a coluna original "gravacao"
tabelaSemDDDnulo = tabelaSemDDDnulo.drop("gravacao", axis=1)

# Concatenação das colunas DDD e Telefone
tabelaSemDDDnulo['ddd_telefone'] = tabelaSemDDDnulo['ddd'].astype(str) + tabelaSemDDDnulo['telefone'].astype(str)

# Identificando qual a região do DDD, através da comparação
tabelaDDD = tabelaDDD.drop_duplicates('cd_ddd')
tabelaSemDDDnulo['regiao'] = tabelaSemDDDnulo['ddd'].map(tabelaDDD.set_index('cd_ddd')['cd_regiao'])

# Inserindo a palavra NULO nos campos sem informação
tabelaSemDDDnulo['regiao'].fillna('NULO', inplace=True)

# Realiza a identificação do Status da Tabulação, através da comparação com a coluna de tabulação
tabelaComparativa = tabelaComparativa.drop_duplicates('tabulacao')
tabelaSemDDDnulo['status'] = tabelaSemDDDnulo['historico_tipo_descricao'].map(tabelaComparativa.set_index('tabulacao')['status'])

# Inserindo a palavra NULO nos campos sem informação
tabelaSemDDDnulo['status'].fillna('NULO', inplace=True)

# Realiza a identificação do Status da Tabulação, através da comparação com a coluna de tabulação
tabelaSemDDDnulo['matricula'] = tabelaSemDDDnulo['usuario_gravou_codigo_descricao'].map(tabelaMOP.set_index('nomeOperador')['matricula'])

# Filtrando a tabela, removendo os campos sem informação
tabelaSemDDDnulo = tabelaSemDDDnulo.loc[tabelaSemDDDnulo['status'] != 'NULO']

qtd_tabelaSemDDDnulo2 = len(tabelaSemDDDnulo)
print(f'Quantidade filtrada, removendo os campos incorretos de Status, ficando com: {qtd_tabelaSemDDDnulo2} registros')

# Chama o método de identificação do último registro importado no banco de dados e acrescenta mais 1
cod_arquivo_atual = valorInicial + 1

# Atribui ID's para todos os registros restantes
tabelaSemDDDnulo['id'] = range(cod_arquivo_atual, cod_arquivo_atual + len(tabelaSemDDDnulo))

ultimalinha = tabelaSemDDDnulo['id'].iloc[-1]

# Cria uma nova tabela com as colunas renomeadas
tabela_renomear = tabelaSemDDDnulo.rename(columns={
    'id': 'ID',
    'data': 'DATA',
    'hora': 'INTERVALO',
    'regiao': 'Regiao',
    'ddd_telefone': 'DDD_NTC',
    'historico_tipo_descricao': 'Tabulacao_1',
    'status': 'Status_Tabulacao',
    'matricula': 'Matricula',
    'usuario_gravou_codigo_descricao': 'NomeOperador'})

# Lista com as colunas a serem mantidas na nova tabela
colunas_mantidas = ['ID', 'DATA', 'INTERVALO', 'Regiao', 'DDD_NTC', 'Tabulacao_1', 'Status_Tabulacao', 'Matricula', 'NomeOperador']

# Criar uma nova tabela mantendo apenas as colunas desejadas
tabela_import = tabela_renomear[colunas_mantidas]

dataRetorno = tabelaSemDDDnulo['data'].iloc[-1]

dataRetorno = dataRetorno.strftime('%d%m%Y')

# Nome do arquivo CSV a ser salvo
nome_arquivo_csv = "RETORNO_TRATADO_DATA_"+dataRetorno+"_"+operation.upper()+".csv"

# Diretório onde deseja salvar o arquivo
diretorio_salvar = '/content/drive/MyDrive/TIMDescargaDeRetorno'

# Caminho completo para salvar o arquivo
caminho_arquivo = os.path.join(diretorio_salvar, nome_arquivo_csv)

# Salvar a tabela em formato CSV
tabela_import.to_csv(caminho_arquivo, index=False)

print("==========================================================================================================================================")

print("Tabela salva como arquivo CSV.")

print(f'O arquivo {nome_do_arquivo}.csv foi tratado e gerado o arquivo {nome_arquivo_csv}')

print(f'O ID da última linha foi: {ultimalinha}')







Mounted at /content/drive
A tabela Bruta possui 106722
A tabela teve filtro de excluindo os campos sem DDD, ficando com a quantidade de: 106640 registros
Quantidade filtrada, removendo os campos incorretos de Status, ficando com: 106421 registros


<ipython-input-8-290cd6f4bf7d>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabelaSemDDDnulo['id'] = range(cod_arquivo_atual, cod_arquivo_atual + len(tabelaSemDDDnulo))


Tabela salva como arquivo CSV.
O arquivo ArquivoBruto_ControleControle_31052024.csv foi tratado e gerado o arquivo RETORNO_TRATADO_DATA_29052024_CONTROLECONTROLE.csv
O ID da última linha foi: 6622013
